Checks whether the most recent dates are the same for each region.

In [1]:
import os
import pandas as pd
import numpy as np

In [2]:
current_dir = os.path.abspath('')  #./ongoing/prescriptors
ROOT_DIR =  os.path.abspath(os.path.join(current_dir, os.pardir))  # ./ongoing
DATA_PATH = os.path.join(ROOT_DIR, 'data')
DATA_FILE_PATH = os.path.join(DATA_PATH, 'OxCGRT_latest1.csv')
ADDITIONAL_CONTEXT_FILE = os.path.join(DATA_PATH, "Additional_Context_Data_Global.csv")
ADDITIONAL_US_STATES_CONTEXT = os.path.join(DATA_PATH, "US_states_populations.csv")
ADDITIONAL_UK_CONTEXT = os.path.join(DATA_PATH, "uk_populations.csv")

In [3]:
NPI_COLUMNS = ['C1_School closing',
               'C2_Workplace closing',
               'C3_Cancel public events',
               'C4_Restrictions on gatherings',
               'C5_Close public transport',
               'C6_Stay at home requirements',
               'C7_Restrictions on internal movement',
               'C8_International travel controls',
               'H1_Public information campaigns',
               'H2_Testing policy',
               'H3_Contact tracing',
               'H6_Facial Coverings']

CONTEXT_COLUMNS = ['CountryName',
                   'RegionName',
                   'GeoID',
                   'Date',
                   'ConfirmedCases',
                   'ConfirmedDeaths',
                   'Population']
NB_LOOKBACK_DAYS = 21
NB_TEST_DAYS = 14
WINDOW_SIZE = 7
US_PREFIX = "United States / "
# NUM_TRIALS = 1
NUM_TRIALS = 10
LSTM_SIZE = 32
MAX_NB_COUNTRIES = 20

In [4]:
# load data
df = pd.read_csv(DATA_FILE_PATH,
                 parse_dates=['Date'],
                 encoding="ISO-8859-1",
                 dtype={"RegionName": str,
                        "RegionCode": str},
                 error_bad_lines=False)
df["GeoID"] = np.where(df["RegionName"].isnull(),
                           df["CountryName"],
                           df["CountryName"] + ' / ' + df["RegionName"])
df

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,GeoID
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-02,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-03,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-04,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-05,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105555,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-07,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe
105556,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-08,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe
105557,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-09,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe
105558,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-10,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe


In [5]:
# load additonal contexts data
# File containing the population for each country
# Note: this file contains only countries population, not regions
additional_context_df = pd.read_csv(ADDITIONAL_CONTEXT_FILE,
                                    usecols=['CountryName', 'Population'])

additional_context_df['GeoID'] = additional_context_df['CountryName']

# US states population
additional_us_states_df = pd.read_csv(ADDITIONAL_US_STATES_CONTEXT,
                                      usecols=['NAME', 'POPESTIMATE2019'])
# Rename the columns to match measures_df ones
additional_us_states_df.rename(columns={'POPESTIMATE2019': 'Population'}, inplace=True)
# Prefix with country name to match measures_df
additional_us_states_df['GeoID'] = US_PREFIX + additional_us_states_df['NAME']

# Append the new data to additional_df
additional_context_df = additional_context_df.append(additional_us_states_df)

# UK population
additional_uk_df = pd.read_csv(ADDITIONAL_UK_CONTEXT)
# Append the new data to additional_df
additional_context_df = additional_context_df.append(additional_uk_df)


In [6]:
additional_context_df

,CountryName,Population,GeoID,NAME
0,Afghanistan,38928346,Afghanistan,NaN
1,Albania,2877797,Albania,NaN
2,Algeria,43851044,Algeria,NaN
3,Andorra,77265,Andorra,NaN
4,Angola,32866272,Angola,NaN
...,...,...,...,...
57,NaN,104914,United States / Virgin Islands,Virgin Islands
0,NaN,55980000,United Kingdom / England,NaN
1,NaN,1882000,United Kingdom / Northern Ireland,NaN
2,NaN,5454000,United Kingdom / Scotland,NaN


In [7]:
def fill_missing_values(df):
    """
    # Fill missing values by interpolation, ffill, and filling NaNs
    :param df: Dataframe to be filled
    """
    df.update(df.groupby('GeoID').ConfirmedCases.apply(
        lambda group: group.interpolate(limit_area='inside')))
    # Drop country / regions for which no number of cases is available
    df.dropna(subset=['ConfirmedCases'], inplace=True)
    df.update(df.groupby('GeoID').ConfirmedDeaths.apply(
        lambda group: group.interpolate(limit_area='inside')))
    # Drop country / regions for which no number of deaths is available
    df.dropna(subset=['ConfirmedDeaths'], inplace=True)
    for npi_column in NPI_COLUMNS:
        df.update(df.groupby('GeoID')[npi_column].ffill().fillna(0))
    return df

In [8]:
def prepare_dataframe(data, add_data) -> pd.DataFrame:
    """
    Loads the Oxford dataset, cleans it up and prepares the necessary columns. Depending on options, also
    loads the Johns Hopkins dataset and merges that in.
    :param data_url: the url containing the original data
    :return: a Pandas DataFrame with the historical data
    """
    # Original df from Oxford
    df1 = data

    # Additional context df (e.g Population for each country)
    df2 = add_data

    # Merge the 2 DataFrames
    df = df1.merge(df2, on=['GeoID'], how='left', suffixes=('', '_y'))

    # Drop countries with no population data
    df.dropna(subset=['Population'], inplace=True)

    #  Keep only needed columns
    columns = CONTEXT_COLUMNS + NPI_COLUMNS
    df = df[columns]

    # Fill in missing values
    df = fill_missing_values(df)
 
    # Compute number of new cases and deaths each day
    df['NewCases'] = df.groupby('GeoID').ConfirmedCases.diff().fillna(0)
    df['NewDeaths'] = df.groupby('GeoID').ConfirmedDeaths.diff().fillna(0)

    # Replace negative values (which do not make sense for these columns) with 0
    df['NewCases'] = df['NewCases'].clip(lower=0)
    df['NewDeaths'] = df['NewDeaths'].clip(lower=0)

    # Compute smoothed versions of new cases and deaths each day
    df['SmoothNewCases'] = df.groupby('GeoID')['NewCases'].rolling(
        WINDOW_SIZE, center=False).mean().fillna(0).reset_index(0, drop=True)
    df['SmoothNewDeaths'] = df.groupby('GeoID')['NewDeaths'].rolling(
        WINDOW_SIZE, center=False).mean().fillna(0).reset_index(0, drop=True)

    # Compute percent change in new cases and deaths each day
    df['CaseRatio'] = df.groupby('GeoID').SmoothNewCases.pct_change(
    ).fillna(0).replace(np.inf, 0) + 1
    df['DeathRatio'] = df.groupby('GeoID').SmoothNewDeaths.pct_change(
    ).fillna(0).replace(np.inf, 0) + 1

    # Add column for proportion of population infected
    df['ProportionInfected'] = df['ConfirmedCases'] / df['Population']

    # Create column of value to predict
    df['PredictionRatio'] = df['CaseRatio'] / (1 - df['ProportionInfected'])

    return df

In [9]:
d = prepare_dataframe(df, additional_context_df)  # preprocess

In [10]:
pd.options.display.max_rows = 4000
d.groupby(["GeoID"])["Date"].max()

GeoID
Afghanistan                         2021-01-10
Albania                             2021-01-10
Algeria                             2021-01-10
Andorra                             2021-01-10
Angola                              2021-01-10
Argentina                           2021-01-10
Aruba                               2021-01-10
Australia                           2021-01-10
Austria                             2021-01-10
Azerbaijan                          2021-01-10
Bahamas                             2021-01-10
Bahrain                             2021-01-10
Bangladesh                          2021-01-10
Barbados                            2021-01-10
Belarus                             2021-01-10
Belgium                             2021-01-10
Belize                              2021-01-10
Benin                               2021-01-10
Bermuda                             2021-01-10
Bhutan                              2021-01-10
Bolivia                             2021-01-10
Bosnia 

In [11]:
d.groupby(["GeoID"])["Date"].max().min()  # get minimum of the maximium dates for each region

Timestamp('2021-01-09 00:00:00')

In [12]:
d.loc[d.GeoID =='United Kingdom / Wales'][['Date', 'ConfirmedCases']]

,Date,ConfirmedCases
39661,2020-03-17,213.0
39662,2020-03-18,249.0
39663,2020-03-19,309.0
39664,2020-03-20,382.0
39665,2020-03-21,457.0
39666,2020-03-22,533.0
39667,2020-03-23,666.0
39668,2020-03-24,807.0
39669,2020-03-25,997.0
39670,2020-03-26,1174.0


In [13]:
# remove date 2021-01-10
idx = df[ df['Date'] == '2021-01-10'].index 
idx

Int64Index([   375,    752,   1129,   1506,   1883,   2260,   2637,   3014,
              3391,   3768,
            ...
            102165, 102542, 102919, 103296, 103673, 104050, 104427, 104804,
            105181, 105558],
           dtype='int64', length=280)

In [14]:
df.drop(index=idx, inplace=True)

In [ ]:
df

In [ ]:
idx = df[ df['Date'] == '2021-01-11'].index 

In [ ]:
idx

In [ ]:
df.loc[idx]

In [ ]:
df.drop(idx, inplace=True)

In [ ]:
#file = os.path.join(DATA_PATH, 'new_file.cs')
df.to_csv("OxCGRT_latest_2021-01-09.csv", index=False)

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,GeoID
0,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-01,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
1,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-02,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
2,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-03,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
3,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-04,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
4,Aruba,ABW,NaN,NaN,NAT_TOTAL,2020-01-05,0.0,NaN,0.0,NaN,...,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.0,Aruba
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
105554,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-06,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe
105555,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-07,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe
105556,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-08,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe
105557,Zimbabwe,ZWE,NaN,NaN,NAT_TOTAL,2021-01-09,2.0,1.0,3.0,1.0,...,92.59,96.43,96.43,71.11,71.11,78.21,78.21,25.0,25.0,Zimbabwe


In [16]:
idx = df[ df['Date'] == '2021-01-11'].index 

In [17]:
idx

Int64Index([   376,    753,   1130,   1507,   1884,   2261,   2638,   3015,
              3392,   3769,
            ...
            102166, 102543, 102920, 103297, 103674, 104051, 104428, 104805,
            105182, 105559],
           dtype='int64', length=280)

In [18]:
df.loc[idx]

,CountryName,CountryCode,RegionName,RegionCode,Jurisdiction,Date,C1_School closing,C1_Flag,C2_Workplace closing,C2_Flag,...,StringencyIndexForDisplay,StringencyLegacyIndex,StringencyLegacyIndexForDisplay,GovernmentResponseIndex,GovernmentResponseIndexForDisplay,ContainmentHealthIndex,ContainmentHealthIndexForDisplay,EconomicSupportIndex,EconomicSupportIndexForDisplay,GeoID
376,Aruba,ABW,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,31.48,NaN,36.43,NaN,40.00,NaN,36.54,NaN,62.5,Aruba
753,Afghanistan,AFG,NaN,NaN,NAT_TOTAL,2021-01-11,0.0,NaN,0.0,NaN,...,8.33,13.10,13.10,20.00,20.00,23.08,23.08,0.0,0.0,Afghanistan
1130,Angola,AGO,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,65.74,NaN,80.48,NaN,48.89,NaN,56.41,NaN,0.0,Angola
1507,Albania,ALB,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,60.19,NaN,67.86,NaN,60.56,NaN,62.18,NaN,50.0,Albania
1884,Andorra,AND,NaN,NaN,NAT_TOTAL,2021-01-11,1.0,1.0,2.0,1.0,...,52.78,59.52,59.52,63.33,63.33,57.69,57.69,100.0,100.0,Andorra
2261,United Arab Emirates,ARE,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,49.07,NaN,54.76,NaN,59.78,NaN,61.28,NaN,50.0,United Arab Emirates
2638,Argentina,ARG,NaN,NaN,NAT_TOTAL,2021-01-11,3.0,1.0,2.0,0.0,...,79.17,79.76,79.76,74.94,74.94,74.94,74.94,75.0,75.0,Argentina
3015,Australia,AUS,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,65.28,NaN,65.00,NaN,65.00,NaN,63.46,NaN,75.0,Australia
3392,Austria,AUT,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,78.70,NaN,84.52,NaN,78.89,NaN,75.64,NaN,100.0,Austria
3769,Azerbaijan,AZE,NaN,NaN,NAT_TOTAL,2021-01-11,NaN,NaN,NaN,NaN,...,89.81,NaN,91.67,NaN,73.33,NaN,82.69,NaN,12.5,Azerbaijan


In [19]:
df.drop(idx, inplace=True)

In [21]:
#file = os.path.join(DATA_PATH, 'new_file.cs')
df.to_csv("OxCGRT_latest_2021-01-09.csv", index=False)